In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 15334557
paper_name = 'mollapour_piper_2004' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
files = ['Table1.txt','Table2.txt','Table3.txt']

In [8]:
original_data_list = []
for ixf, f in enumerate(files):
    original_data = pd.read_csv('raw_data/' + f, header=None, names=['orf'])
    print('Original data dimensions: %d x %d' % (original_data.shape))
    print(original_data.head())
    original_data['orf'] = original_data['orf'].astype(str)
    original_data['orf'] = clean_orf(original_data['orf'])
    
    typo_fixes = {'TMR205C':'YMR205C','YMLO48W':'YML048W',
                  'YLRO25W':'YLR025W','YKLO54W':'YKL054W',
                  'YCLO58C':'YCL058C'}
    original_data['orf'] = original_data['orf'].apply(lambda x: typo_fixes[x] if x in typo_fixes.keys() else x)
    
    original_data['orf'] = translate_sc(original_data['orf'], to='orf')
    t = looks_like_orf(original_data['orf'])
    print(original_data.loc[~t,])
    
    if ixf == 2:
        original_data['data'] = 1
    else:
        original_data['data'] = -1
        
    original_data.set_index('orf', inplace=True)
    original_data = original_data[['data']].copy()
    original_data = original_data.groupby(original_data.index).mean()
    
    print(original_data.shape)
    
    original_data_list.append(original_data)

Original data dimensions: 12 x 1
       orf
0  YCR047c
1  YOL051w
2  YJR090c
3  YOR038c
4  YLR244c
Empty DataFrame
Columns: [orf]
Index: []
(12, 1)
Original data dimensions: 237 x 1
       orf
0  YGR240c
1  TMR205c
2  YDR074w
3  YHR183w
4  YMLO48w
Empty DataFrame
Columns: [orf]
Index: []
(234, 1)
Original data dimensions: 56 x 1
       orf
0  YPL203W
1  YKL011C
2  YBR227C
3  YKL085W
4  YLR061W
Empty DataFrame
Columns: [orf]
Index: []
(56, 1)


In [9]:
original_data = pd.concat(original_data_list, axis=1)

In [11]:
original_data[original_data.isnull()] = 0

In [20]:
original_data.index.name = 'orf'

In [21]:
original_data.head()

,data,data,data
orf,,,
YCR047C,-1.0,0.0,0.0
YJR090C,-1.0,-1.0,0.0
YLR118C,-1.0,0.0,0.0
YLR240W,-1.0,-1.0,0.0
YLR244C,-1.0,0.0,0.0


In [22]:
original_data.shape

(299, 3)

# Prepare the final dataset

In [23]:
data = original_data.copy()

In [24]:
dataset_ids = [487, 554, 555]
datasets = datasets.reindex(index=dataset_ids)

In [25]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [26]:
data.head()

dataset_id,487,554,555
data_type,value,value,value
orf,,,
YCR047C,-1.0,0.0,0.0
YJR090C,-1.0,-1.0,0.0
YLR118C,-1.0,0.0,0.0
YLR240W,-1.0,-1.0,0.0
YLR244C,-1.0,0.0,0.0


## Subset to the genes currently in SGD

In [27]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 13


In [28]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

data.head()

,dataset_id,487,554,555
,data_type,value,value,value
gene_id,orf,,,
604,YCR047C,-1.0,0.0,0.0
3551,YJR090C,-1.0,-1.0,0.0
3807,YLR118C,-1.0,0.0,0.0
3929,YLR240W,-1.0,-1.0,0.0
3933,YLR244C,-1.0,0.0,0.0


# Normalize

In [31]:
data_norm = normalize_phenotypic_scores(data, has_tested=False)

In [32]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [33]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

data_all.head()

,dataset_id,487,554,555,487,554,555
,data_type,value,value,value,valuez,valuez,valuez
gene_id,orf,,,,,,
604,YCR047C,-1.0,0.0,0.0,-19.529892,0.000000,0.0
3551,YJR090C,-1.0,-1.0,0.0,-19.529892,-4.550869,0.0
3807,YLR118C,-1.0,0.0,0.0,-19.529892,0.000000,0.0
3929,YLR240W,-1.0,-1.0,0.0,-19.529892,-4.550869,0.0
3933,YLR244C,-1.0,0.0,0.0,-19.529892,0.000000,0.0


# Print out

In [34]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [35]:
from IO.save_data_to_db3 import *

In [36]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/3 [00:00<?, ?it/s]

Deleting all datasets for PMID 15334557...
Inserting the new data...


100%|██████████| 3/3 [00:01<00:00,  1.87it/s]

Updating the data_modified_on field...
